## Jupyter에서 가상 환경 인식시키는 방법 
1. 폴더 생성 후 가상환경을 생성

   ```bash
   mkdir llm_project
   cd llm_project
    
   # git 저장소 복사
   git clone https://github.com/kdu9303/news_classifier_model.git
    
   # 가상환경 생성
   python -m venv venv
    
   # 3.10버전으로 설치
   python3.10 -m venv venv

   ```
3. 가상환경 activate 후 requirements.txt를 통해 패키지 설치

   ```bash
   pip install -r requirements.txt --upgrade
    mkdir llm_project
    cd llm_project
   mkdir llm_project
   cd llm_project
    
   # 가상환경 생성
   python -m venv venv
    
   # 3.10버전으로 설치
   python3.10 -m venv venv
   ```
3. 가상환경 activate 후 requirements.txt를 통해 패키지 설치

   ```bash
    pip install -r requirements.txt --upgrade
    ```
5. 가상환경과 Jypyter kernel 연결 

   ```bash
   # python -m ipykernel install --user --name "가상환경이름" --display-name "노트북에 표시할 이름"
   python -m ipykernel install --user --name venv --display-name llm_venv
    ```
7. Jupyter lab 실행

   ```bash
   jupyter lab
    ```   pip install -r requirements.txt --upgrade
   ```
5. 가상환경과 Jypyter kernel 연결 

   ```bash
   # python -m ipykernel install --user --name "가상환경이름" --display-name "노트북에 표시할 이름"
   python -m ipykernel install --user --name venv --display-name llm_venv
   ```
7. Jupyter lab 실행

   ```bash
   jupyter lab
   ```

In [1]:
%config Completer.use_jedi = False

# 패키지 Import


In [2]:
import os
import time
import glob
import warnings
import aiocsv
import aiofiles
import tiktoken
import traceback
import pandas as pd
from rich import print
from functools import wraps
from typing import Any, Coroutine, Generator, AsyncGenerator
from dotenv import load_dotenv

# text 전처리
from langchain_core.documents import Document
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader,
)
from langchain_community.document_loaders.excel import UnstructuredExcelLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

# embeddings
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.prompts import PromptTemplate

# LLM
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.chains import (
    create_retrieval_chain,
)

load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

## 모델 클래스 정의
- 훈련 데이터의 폴더와 Input용 데이터가 담긴 폴더의 경로를 인자로 넘겨야합니다.
- Input 형태는 csv와 xlsx 엑셀파일만 가능합니다.
    - 텍스트 형태의 데이터는 기사 단위(row단위)로 프로세스가 어려우므로 제외
- 사용 가능 모델
    - gpt-3.5-turbo
    - gpt-4-turbo
    - claude-2.1
    - claude-3-haiku-20240307
- 훈련 데이터 가능 확장자
    - pdf, docx, txt, xlsx, csv


In [3]:
def async_timer(func):
    @wraps(func)
    async def wrapper(*args, **kwargs):
        start_time = time.perf_counter()

        # call the decorated function
        result = await func(*args, **kwargs)

        end_time = time.perf_counter()
        execution_time = end_time - start_time

        if func.__name__ == "main":
            print(f"----------------------------------------")
            print(f"---------- Main 함수 실행 --------------")
            print(f"총 실행 시간: {execution_time:.3f}초")
            print(f"----------------------------------------")
        else:
            print(f"----------------------------------------")
            print(f"실행 시간: {execution_time:.3f}초")
            print(f"----------------------------------------")
        return result

    return wrapper


class LlmArticleClassifier:
    def __init__(self, train_dir_path: str, model: str = "gpt-3.5-turbo"):
        self.train_dir_path = train_dir_path

        self.model = model

    async def async_get_input_data_from_files(
        self, input_dir_path
    ) -> AsyncGenerator[str, None]:
        for file_name in glob.glob(input_dir_path + "/*"):
            try:
                if ".xlsx" in file_name:
                    data = pd.read_excel(file_name)
                elif ".csv" in file_name:
                    data = pd.read_csv(file_name)
                else:
                    continue

                for _, row in data.iterrows():
                    row_dict = row.to_dict()
                    row_str = "\n".join(f'"{k}": "{v}"' for k, v in row_dict.items())
                    yield row_str

            except Exception:
                print(traceback.format_exc())

    def sync_get_input_data_from_files(self) -> Generator[str, None, None]:
        # formatted_data_strings = []

        for file_name in glob.glob(self.input_dir_path + "/*"):
            try:
                if ".xlsx" in file_name:
                    data = pd.read_excel(file_name)
                elif ".csv" in file_name:
                    data = pd.read_csv(file_name)

                for _, row in data.iterrows():
                    row_dict = row.to_dict()
                    row_str = "\n".join(f'"{k}": "{v}"' for k, v in row_dict.items())
                    # formatted_data_strings.append(row_str)
                    yield row_str

            except Exception as e:
                print(traceback.format_exc())

        # return formatted_data_strings

    def get_train_data_from_files(self) -> list[Document]:
        doc_list = []

        for file_name in glob.glob(self.train_dir_path + "/*"):
            print(f"Uploaded {file_name}")

            if ".pdf" in file_name:
                loader = PyPDFLoader(file_name)
                documents = loader.load_and_split()
            elif ".docx" in file_name:
                loader = Docx2txtLoader(file_name)
                documents = loader.load_and_split()
            elif ".txt" in file_name:
                loader = TextLoader(file_name)
                documents = loader.load_and_split()
            elif ".xlsx" in file_name:
                loader = UnstructuredExcelLoader(file_name, mode="elements")
                documents = loader.load_and_split()
            elif ".csv" in file_name:
                loader = CSVLoader(file_name)
                documents = loader.load_and_split()
            else:
                continue

            doc_list.extend(documents)

        return doc_list

    def tiktoken_len(self, text: str) -> int:
        tokenizer = tiktoken.get_encoding("cl100k_base")
        tokens = tokenizer.encode(text)
        return len(tokens)

    def get_text_chunks(self) -> list[Document]:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1024, chunk_overlap=100, length_function=self.tiktoken_len
        )
        text = self.get_train_data_from_files()
        chunks = text_splitter.split_documents(text)
        return chunks

    async def get_vectorstore(self) -> Coroutine[Any, Any, FAISS]:
        embeddings = HuggingFaceEmbeddings(
            model_name="jhgan/ko-sroberta-multitask",
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
        text_chunks = self.get_text_chunks()
        vectordb = await FAISS.afrom_documents(text_chunks, embeddings)
        return vectordb

    def set_templetes(self):
        document_template = f"""
            너의 역할은 주어진 뉴스 기사나 리포트를 분석하고 제공된 Output 형식의 카테고리 분류 설명에 따라 
            입력 기사를 카테고리별로 분류하는 역할이야.
            결과를 출력할때는 Markdown에서 글씨를 보기 쉽게 문장 혹은 단위별로 prettify를 적용해서 출력해줘.
            
            리포트 혹은 뉴스의 내용과 Output 형식은 백틱(```)안에 주어진다.
            예제 리포트에 포함된 내용은 결과에 Return하지 않는다.
            
            예제 리포트:
            {{context}}
            
            ```
            {{input}}
            ```
            """
        document_prompt = PromptTemplate(
            template=document_template, input_variables=["input"]
        )

        return document_prompt

    async def get_create_retrieval_chain(self, temperature: float = 0.0):

        if "gpt" in self.model:
            llm = ChatOpenAI(
                api_key=os.getenv("open_ai_key"),
                model_name=self.model,
                temperature=temperature,
            )
        elif "claude" in self.model:
            llm = ChatAnthropic(
                api_key=os.getenv("antrophic_key"),
                model_name=self.model,
                temperature=temperature,
            )

        """
        Retriever 옵션
        as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})
        as_retriever(search_kwargs={"k": 3})
        as_retriever(search_type="mmr")
        """
        vectorstore = await self.get_vectorstore()
        retriever = vectorstore.as_retriever()
        document_prompt = self.set_templetes()

        combine_docs_chain = create_stuff_documents_chain(llm, document_prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

        return retrieval_chain

    @async_timer
    async def process_llm(self, query: str) -> str:
        chain = await self.get_create_retrieval_chain()
        result = await chain.ainvoke({"input": query})
        print(result["answer"])

        return result

    @async_timer
    async def process_llm_batch(self, queries: list[str]) -> list[dict[str, str]]:
        chain = await self.get_create_retrieval_chain()
        results = await chain.abatch([{"input": query} for query in queries])
        for result in results:
            # print(f"Query: {result['input']}")
            print(result['answer'])
            print()
        return results    


### CSV 파일 저장

In [4]:
async def write_to_csv(results: list[dict[str, str]]):

    output_dir = "./output_data"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, "results.csv")

    async with aiofiles.open(
        output_file, mode="w", encoding="utf-8-sig", newline=""
    ) as afp:
        writer = aiocsv.AsyncWriter(afp, dialect="unix")
        # header
        await writer.writerow(["query", "answer"])

        # rows
        for result in results:
            await writer.writerow([result["input"], result["answer"]])

    print(f"결과가 '{output_file}'에 저장되었습니다.")

# main 함수
main 함수에서는 LlmArticleClassifier 클래스를 초기화하고, 입력 데이터를 가져와서 process_llm 메서드를 실행하는 방식으로 사용됩니다. 코드에서는 비동기 방식으로 데이터를 처리하고, 일정한 크기의 청크로 나누어 병렬로 처리하는 방식을 사용하고 있습니다.

### main 함수의 동작 과정:

1. LlmArticleClassifier 클래스를 초기화
   - train_dir_path -> 훈련용 데이터 폴더 위치를 인자로 전달
2. LlmArticleClassifier.async_get_input_data_from_files()
   - 폴더에 있는 파일의 row단위를 input형태로 변환
   - input_dir_path -> Input 폴더의 위치를 인자로 전달
3. data_generator로부터 일정한 크기의 묶음으로 나누어 병렬로 처리
   - chunk -> 병렬처리용 Thread 개수를 설정
4. 각 청크에 대해 병렬로 결과 처리
   - write_to_csv



In [5]:
@async_timer
async def main():
    llm = LlmArticleClassifier(train_dir_path="./train_data", model="gpt-3.5-turbo")
    
    # 폴더 내의 모든 엑셀파일 인식
    data_generator = llm.async_get_input_data_from_files(
        input_dir_path="./input_data",
    )

    results = []
    chunk_size = 4  # 한 번에 처리할 chunk 크기
    batch_queries = []
    async for query in data_generator:
        batch_queries.append(query)
        if len(batch_queries) == chunk_size:
            results.extend(await llm.process_llm_batch(batch_queries))
            batch_queries.clear()

    # 잔여 task가 있을경우 처리
    if batch_queries:
        results.extend(await llm.process_llm_batch(batch_queries))
    
    await write_to_csv(results)

In [6]:
await main()

Uploaded ./train_data/training_data.txt

제목: '메가존클라우드, 데이터브릭스 최고 등급 ‘엘리트’ 파트너 선정'
언론사: 한국경제TV
발행일자: 2024-05-08 09:57:28
대상기업: 메가존클라우드
내용: 
    - 내용 소제목: 데이터브릭스 최고 등급 ‘엘리트’ 파트너 선정
    - 내용 요약: 메가존클라우드가 데이터브릭스의 최고 등급 '엘리트' 파트너로 선정되었으며, 이는 고객의 성공적인 
디지털 전환을 이끈 파트너에 부여되는 등급이다.
        - 수치 요약1: 
            - 수치 내용: 투자
            - 수치: 2000억원
        - 수치 요약2: 
            - 수치 내용: 기업가치
            - 수치: 8000억원
    - 내용 소제목: 파트너십 협력
    - 내용 요약: 메가존클라우드는 데이터브릭스와 파트너십을 체결하여 데이터브릭스 솔루션을 기반으로 기술검증, 
고객사 대상 핸즈온 세션 지원, 데이터 전환 사업 구축 등을 진행하고 있으며, 최고 파트너 등급 '엘리트' 선정을 통해 
데이터와 AI의 활용을 가속화할 계획이다.
    - 내용 소제목: 회사 배경
    - 내용 요약: 메가존클라우드는 2700여명의 클라우드 전문 인력을 보유하고 있으며, 국내 클라우드 MSP업계 최초의 
유니콘 기업으로 등극하였고, 2023년에는 1조5106억원의 매출을 달성하였다.해외에도 미국, 일본, 캐나다, 호주, 중국 
상해·북경, 베트남, 홍콩, 싱가포르 등 8개국에서 현지 법인을 운영하고 있다.
    - 내용 소제목: 파트너십 강화
    - 내용 요약: 메가존클라우드와 데이터브릭스는 더 많은 한국 기업들이 데이터브릭스 데이터 인텔리전스 플랫폼을 
활용할 수 있도록 파트너십을 더욱 강화할 예정이다.

제목: 'KT스카이라이프, 1분기 영업익 29억 원…전년比 82%↓'
언론사: 서울경제
발행일자: 2024-05-08 09:56:03
대상기업: KT스카이라이프
내용: 
    - 내용 소제목: 1분기 영업실적
    - 내용 요약: KT스카이라이프는 올해 1분기에 대폭 축소된 영업이익을 기록하며 부진한 실적을 보였다. 매출액은 소폭 
증가했지만 이익은 크게 감소했다.
        - 수치 요약1: 
            - 수치 내용: 영업이익
            - 수치: 29억 원
        - 수치 요약2: 
            - 수치 내용: 매출액
            - 수치: 2544억 원
        - 수치 요약3: 
            - 수치 내용: 당기순이익
            - 수치: 13억 원
    - 내용 소제목: 영업이익 감소 이유
    - 내용 요약: 영업이익이 줄어든 이유는 콘텐츠 투자 비용 증가와 방송 발전 기금, 프로그램 사용료 증가 등이 영향을 
끼쳤다.
    - 내용 소제목: 매출액 증가
    - 내용 요약: 매출액이 증가한 이유는 인터넷과 모바일 가입자의 지속적인 증가로 전체 유지 가입자 수가 확보되었기 
때문이다.
    - 내용 소제목: ENA 성장
    - 내용 요약: KT스카이라이프의 방송 채널인 ENA가 성장하며 시청률 순위에서 상승하고 광고 수익도 증가하고 있다.

```
제목: "'인공지능?' '아니 공감지능!'…LG 올레드 에보, 50년 전 김환기 작품 소환"
언론사: 데일리안
발행일자: 2024-05-02 10:00:10
대상기업: LG전자
내용: 
    - 내용 소제목: LG 올레드 에보 소개
    - 내용 요약: LG전자의 '공감지능(AI)' TV 'LG 올레드 에보'가 한국 대표 미술가 고(故) 김환기 화백의 작품을 미국 
뉴욕에 선보이는 '디지털 캔버스'로 활약하고 있다. 김환기의 대표작을 재해석한 미디어아트가 전시되며, LG 올레드 에보는
AI 화질‧음질 프로세서인 '알파11'을 탑재하여 사용자에게 차별화된 고객경험을 제공한다.
    - 내용 소제목: 뉴욕 전시
    - 내용 요약: LG전자는 '프리즈 뉴욕'에서 추상미술의 거장 김환기 화백의 작품을 재해석한 미디어아트 5점을 2024년형
LG 올레드 에보로 선보인다. 김환기의 작품을 미디어아트로 표현하며, 뉴욕한국문화원에서도 LG 올레드 에보를 통해 김환기
작품 관련 미디어아트가 전시된다.
    - 내용 소제목: LG 올레드 에보 기능
    - 내용 요약: LG 올레드 에보는 AI 화질‧음질 프로세서 '알파11'을 탑재하여 사용자의 취향을 스스로 파악하고 공간 
사운드를 풍성하게 제공한다. AI 챗봇 기능을 통해 사용자의 목소리를 구별하고 맞춤 콘텐츠 추천 및 화질 설정을 
제공한다.
```
```

제목: '팹리스 업체' 퓨리오사AI, 2000억 투자 유치 추진
언론사: thebell
발행일자: 20230119 08:05:04
대상기업: 퓨리오사AI
내용: 
    - 내용 소제목: 퓨리오사AI, 최대 2000억 투자 유치 추진
    - 내용 요약: 퓨리오사AI가 최대 2000억원 규모의 투자 유치를 추진 중이며, 기업가치는 8000억원 이상이다. 산은과 
DSC인베스트먼트가 투자에 참여할 예정이며, 후속 투자 가능성이 열려있다.
        - 수치 요약1: 
            - 수치 내용: 시리즈C 투자 유치 규모
            - 수치: 2000억원
        - 수치 요약2: 
            - 수치 내용: 기업가치
            - 수치: 8000억원
        - 수치 요약3: 
            - 수치 내용: 이전 유치 규모
            - 수치: 800억원

```
'국제민간항공기구-에콰도르 키토공항과 협력확대'
생성일자: 2024-05-02 10:00:03
세부분류: 산업/재계
언론사: 파이낸셜뉴스
url: https://n.news.naver.com/mnews/article/014/0005179679
기사_내용: 인천국제공항공사와 국제민간항공기구(ICAO)가 협력을 확대하는 내용이다. 이번 MOU 체결을 통해 공사는 
직원들의 해외 직무수행 경험 기회를 확대하여 향후 해외사업 전문 인력으로 육성한다는 방침이다.
```

...

결과가 './output_data/results.csv'에 저장되었습니다.

----------------------------------------

---------- Main 함수 실행 --------------

총 실행 시간: 24.413초

----------------------------------------